In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
import json
from lsst_efd_client import EfdClient

In [ ]:
# This file needs to be manually downloaded from RubinTV
filename = '/home/craiglagegit/DATA/auxtel_anemom_01feb23/auxtel_2023-02-01.json'
df = pd.read_json(filename)
df = df.transpose()
limit = 0.25 # Limit in arcseconds
df_bad = df[df['Mount motion image degradation'] > limit]
df_good = df[df['Mount motion image degradation'] <= limit]
mount_degradation = df['Mount motion image degradation']
total = np.count_nonzero(~pd.isna(mount_degradation.values))
print(f" There are {len(df_bad)} values > {limit} out of {total} values")

In [ ]:
columns_to_keep = ['Altitude', 'Azimuth', 'Exposure id', 'Mount motion image degradation', 'TAI']
df_bad = df_bad[columns_to_keep]
df_bad['Type'] = 'Other'
df_good = df_good[columns_to_keep]

## Now we manually look at the plots to determine the failure type

In [ ]:
# 
for index in df_bad.index.to_list():
    type = input(f"Which type is {index}? (C = Cogging, S = Stiction, W = Wind, O = Other")
    if type == "" or type == 'O':
        df_bad.loc[index]['Type'] = 'Other'
    if type == 'C':
        df_bad.loc[index]['Type'] = 'Cogging'
    if type == 'S':
        df_bad.loc[index]['Type'] = 'Stiction'
    if type == 'W':
        df_bad.loc[index]['Type'] = 'Wind'


In [ ]:
# Pickle the dataframe so I don't have to categorize them again
file = open('/home/craiglagegit/DATA/auxtel_anemom_01feb23/df.pkl', 'wb')
pkl.dump(df_bad, file)
file.close()


In [ ]:
# Now unpickle them
file = open('/home/craiglagegit/DATA/auxtel_anemom_01feb23/df.pkl', 'rb')
df_bad = pkl.load(file)
file.close()

## Now we plot the mount fails against the anemometer wind speed

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
# Times to start looking at data
start = Time("2023-02-02 00:20:00Z", scale='utc')
end = Time("2023-02-02 08:00:00Z", scale='utc')

In [ ]:
u = await client.select_time_series('lsst.sal.ESS.airTurbulence', \
                                                ['*'],  start, end)

In [ ]:
# the "speedMagnitude' values in u are not right, so
# We create a new column with the correct values
u['uMag'] = u.apply(lambda row: np.sqrt(row.speed0*row.speed0 + row.speed1*row.speed1 + row.speed2*row.speed2), axis=1)
u['sdMag'] = u.apply(lambda row: np.sqrt(row.speedStdDev0*row.speedStdDev0 + row.speedStdDev1*row.speedStdDev1 + row.speedStdDev2*row.speedStdDev2), axis=1)

In [ ]:
# Categorize mount failures against wind speed

uMagValues = u['speedMagnitude'].values
timestamps = u['timestamp'].values
uMagsGood = []
uMagsWind = []
uMagsStiction = []
mountValsGood = []
mountValsWind = []
mountValsStiction = []
for index in df_good.index.to_list():
    if int(df_good['TAI'].loc[index].split(':')[0]) > 8:
        obs_time = Time("2023-02-01T"+df_good['TAI'].loc[index], scale='tai')
    else:
        obs_time = Time("2023-02-02T"+df_good['TAI'].loc[index], scale='tai')
    mountVal = df_good['Mount motion image degradation'].loc[index]
    uMag = np.interp(obs_time.unix_tai, timestamps, uMagValues)
    if np.isnan(mountVal):
        continue
    mountValsGood.append(mountVal)
    uMagsGood.append(uMag)
for index in df_bad[df_bad['Type']=='Wind'].index.to_list():
    if int(df_bad[df_bad['Type']=='Wind']['TAI'].loc[index].split(':')[0]) > 8:
        obs_time = Time("2023-02-01T"+df_bad[df_bad['Type']=='Wind']['TAI'].loc[index], scale='tai')
    else:
        obs_time = Time("2023-02-02T"+df_bad[df_bad['Type']=='Wind']['TAI'].loc[index], scale='tai')
    mountVal = df_bad[df_bad['Type']=='Wind']['Mount motion image degradation'].loc[index]
    uMag = np.interp(obs_time.unix_tai, timestamps, uMagValues)
    if np.isnan(mountVal):
        continue
    mountValsWind.append(mountVal)
    uMagsWind.append(uMag)
for index in df_bad[df_bad['Type']=='Stiction'].index.to_list():
    if int(df_bad[df_bad['Type']=='Stiction']['TAI'].loc[index].split(':')[0]) > 8:
        obs_time = Time("2023-02-01T"+df_bad[df_bad['Type']=='Stiction']['TAI'].loc[index], scale='tai')
    else:
        obs_time = Time("2023-02-02T"+df_bad[df_bad['Type']=='Stiction']['TAI'].loc[index], scale='tai')
    mountVal = df_bad[df_bad['Type']=='Stiction']['Mount motion image degradation'].loc[index]
    uMag = np.interp(obs_time.unix_tai, timestamps, uMagValues)
    if np.isnan(mountVal):
        continue
    mountValsStiction.append(mountVal)
    uMagsStiction.append(uMag)


In [ ]:
# Now plot the results.  Wind fails only occur at high wind speeds
fig = plt.figure(figsize = (12,4))
plt.suptitle("Mount fails 20230202")
plt.subplots_adjust(wspace=0.5)
plt.subplot(1,2,1)
plt.scatter(uMagsGood, mountValsGood, color='green', label='Good')
plt.scatter(uMagsWind, mountValsWind, color='red', label='Wind')
plt.ylabel("Mount motion image degradation")
plt.xlabel("Anemometer SpeedMag, (units?)")
plt.legend()
plt.subplot(1,2,2)
plt.scatter(uMagsGood, mountValsGood, color='green', label='Good')
plt.scatter(uMagsStiction, mountValsStiction, color='orange', label='Stiction')
plt.ylabel("Mount motion image degradation")
plt.xlabel("Anemometer SpeedMag, (units?)")
plt.legend()

plt.savefig('/home/craiglagegit/DATA/auxtel_anemom_01feb23/Speed_vs_Fails_20230202.png')

In [ ]:
# Now plot when the failures occurred vs wind speed
fig = plt.figure(figsize = (16,8))
plt.suptitle("Mount fails 20230202")
plt.subplots_adjust(hspace=0.5)
plt.subplot(2,1,1)
ax1 = u['speedMagnitude'].plot()
for index in df_bad[df_bad['Type'] == 'Wind'].index.to_list():
    bad_time = Time("2023-02-02T"+df_bad['TAI'].loc[index], scale='tai').isot  
    ax1.axvline(bad_time, color="red")
ax1.axvline(bad_time, color="red", label= "Wind")
for index in df_bad[df_bad['Type'] == 'Stiction'].index.to_list():
    bad_time = Time("2023-02-02T"+df_bad['TAI'].loc[index], scale='tai').isot  
    ax1.axvline(bad_time, color="orange")
ax1.axvline(bad_time, color="orange", label="Stiction")
ax1.legend()
plt.subplot(2,1,2)
ax2 = u['speedMagnitude'].plot()
for index in df_good.index.to_list():
    if int(df_good['TAI'].loc[index].split(':')[0]) > 8:
        continue
    good_time = Time("2023-02-02T"+df_good['TAI'].loc[index], scale='tai').isot  
    ax2.axvline(good_time, color="green")
ax2.axvline(good_time, color="green", label="Good")
ax2.legend()
plt.savefig('/home/craiglagegit/DATA/auxtel_anemom_01feb23/Time_of_Fails_vs_20230202.png')


## In this section, we are trying to understand why the wind speed is varying during the night.
## It depends on where the telescope is pointing.

In [ ]:
# Times to start looking at data
start = Time("2023-02-03 04:39:00Z", scale='utc')
end = Time("2023-02-03 08:55:00Z", scale='utc')

In [ ]:
u = await client.select_time_series('lsst.sal.ESS.airTurbulence', \
                                                ['*'],  start, end)
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                      ['azimuthCalculatedAngle'],  start, end)
el = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                      ['elevationCalculatedAngle'],  start, end)


In [ ]:
plt.subplots_adjust(hspace=0.5)
plt.suptitle(f"Anemom Speed2 vs Azimuth 20230203")
plt.subplot(2,1,1)
u['speed2'].plot()
plt.ylabel("Speed2 (m/s)")
plt.subplot(2,1,2)
az['azimuthCalculatedAngle'].plot()
plt.ylabel("azimuth(degrees)")
plt.savefig('/home/craiglagegit/DATA/auxtel_anemom_01feb23/Speed_vs_Az_20230203.png')

In [ ]:
u.columns

In [ ]:
u['uMag'] = u.apply(lambda row: np.sqrt(row.speed0*row.speed0 + row.speed1*row.speed1 + row.speed2*row.speed2), axis=1)

In [ ]:
u['uMag'].plot()
u['speedMagnitude'].plot()

In [ ]:
plt.scatter(u['uMag'], u['speedMagnitude'])